# Rent the Runway: Combined Model

In [1]:
#Data cleaning
import numpy as np 
import pandas as pd

#Modeling 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

np.random.seed(42)

#Warnings - to keep the notebook clean 
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns= 5000

In [2]:
#Import the dummied numerical columns 
num = pd.read_csv('./rent_the_runway_cleaned_with_dummies.csv')

numeric_cols = num.select_dtypes(include=['int64'])
numeric_cols.dropna(inplace=True)
numeric_cols.head()

,age,fit,item_id,rating,size,user_id,weight,body_type_apple,body_type_athletic,body_type_full bust,body_type_hourglass,body_type_pear,body_type_petite,body_type_straight & narrow,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_vacation,rented_for_wedding,rented_for_work,category_ballgown,category_blazer,category_blouse,category_blouson,category_bomber,category_caftan,category_cami,category_cape,category_cardigan,category_coat,category_combo,category_culotte,category_culottes,category_down,category_dress,category_duster,category_frock,category_gown,category_henley,category_hoodie,category_jacket,category_jeans,category_jumpsuit,category_kaftan,category_kimono,category_knit,category_legging,category_leggings,category_maxi,category_midi,category_mini,category_overalls,category_pant,category_pants,category_parka,category_peacoat,category_poncho,category_print,category_pullover,category_romper,category_sheath,category_shift,category_shirt,category_shirtdress,category_skirt,category_skirts,category_skort,category_suit,category_sweater,category_sweatershirt,category_sweatshirt,category_t-shirt,category_tank,category_tee,category_tight,category_top,category_trench,category_trouser,category_trousers,category_tunic,category_turtleneck,category_vest
0,36,1,815826,10,20,334577,137,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,34,1,1636171,10,8,634115,125,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,0,438881,10,13,988705,124,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32,1,1392841,10,16,977884,158,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,34,2,160612,8,8,795673,135,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
#Import the text column
text = pd.read_csv('./rent_the_runway_text.csv')
text.dropna(inplace=True)
text.head()

,full_review
0,good fit great style comfortable yet elegant r...
1,love the fit and fabric this dress wa perfect ...
2,simple black dress loved the ruffle wore this ...
3,one of my fav rental loved this fun dress the ...
4,pretty but not meant for small chested girl pr...


In [4]:
# Instantiate a CountVectorizer
cvec = CountVectorizer()

# Fit the vectorizer on our corpus
cvec.fit(text['full_review'])

text_data = cvec.transform(text['full_review'])

#another way to see the matrix 
text_data.todense()

text_cols = pd.DataFrame(text_data.toarray(), columns=cvec.get_feature_names())

print(text_cols.shape)

(29221, 17135)


In [5]:
text_cols.head()

aa  aaa  aahhed  aalll  aand  ab  aback  abalone  abandon  abdomen  \
0   0    0       0      0     0   0      0        0        0        0   
1   0    0       0      0     0   0      0        0        0        0   
2   0    0       0      0     0   0      0        0        0        0   
3   0    0       0      0     0   0      0        0        0        0   
4   0    0       0      0     0   0      0        0        0        0   

   abdominal  abfab  abide  ability  abilitycomfort  able  abnormal  \
0          0      0      0        0               0     0         0   
1          0      0      0        0               0     0         0   
2          0      0      0        0               0     0         0   
3          0      0      0        0               0     0         0   
4          0      0      0        0               0     0         0   

   abnormally  aboslutely  about  above  abovetheknee  absentmindedly  \
0           0           0      1      0             0               0   
1           0           0      0      0             0               0   
2           0           0      0      1             0               0   
3           0           0      0      0             0               0   
4           0           0      0      0             0               0   

   absolute  absolutely  absolutley  absolutly  absorb  absorbed  absoulutely  \
0         0           0           0          0       0         0            0   
1         0           0           0          0       0         0            0   
2         0           0           0          0       0         0            0   
3         0           0           0          0       0         0            0   
4         0           0           0          0       0         0            0   

   absoutely  abstract  absurd  abt  abundant  ac  academy  accencuate  \
0          0         0       0    0         0   0        0           0   
1          0         0       0    0         0   0        0           0   
2          0         0       0    0         0   0        0           0   
3          0         0       0    0         0   0        0           0   
4          0         0       0    0         0   0        0           0   

   accent  accented  accenting  accentuate  accentuated  accentuates  \
0       0         0          0           0            0            0   
1       0         0          0           0            0            0   
2       0         0          0           0            0            0   
3       0         0          0           0            0            0   
4       0         0          0           0            0            0   

   accentuating  accentuation  accentued  accenuated  accept  acceptable  \
0             0             0          0           0       0           0   
1             0             0          0           0       0           0   
2             0             0          0           0       0           0   
3             0             0          0           0       0           0   
4             0             0          0           0       0           0   

   accepted  accepting  accesories  accesorize  access  accessoriesblaze  \
0         0          0           0           0       0                 0   
1         0          0           0           0       0                 0   
2         0          0           0           0       0                 0   
3         0          0           0           0       0                 0   
4         0          0           0           0       0                 0   

   accessoriesheels  accessoriesi  accessoriesshoes  accessoriesshoesmakeup  \
0                 0             0                 0                       0   
1                 0             0                 0                       0   
2                 0             0                 0                       0   
3                 0             0                 0                       0   
4                 0     

In [6]:
numeric_cols.head()

,age,fit,item_id,rating,size,user_id,weight,body_type_apple,body_type_athletic,body_type_full bust,body_type_hourglass,body_type_pear,body_type_petite,body_type_straight & narrow,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_vacation,rented_for_wedding,rented_for_work,category_ballgown,category_blazer,category_blouse,category_blouson,category_bomber,category_caftan,category_cami,category_cape,category_cardigan,category_coat,category_combo,category_culotte,category_culottes,category_down,category_dress,category_duster,category_frock,category_gown,category_henley,category_hoodie,category_jacket,category_jeans,category_jumpsuit,category_kaftan,category_kimono,category_knit,category_legging,category_leggings,category_maxi,category_midi,category_mini,category_overalls,category_pant,category_pants,category_parka,category_peacoat,category_poncho,category_print,category_pullover,category_romper,category_sheath,category_shift,category_shirt,category_shirtdress,category_skirt,category_skirts,category_skort,category_suit,category_sweater,category_sweatershirt,category_sweatshirt,category_t-shirt,category_tank,category_tee,category_tight,category_top,category_trench,category_trouser,category_trousers,category_tunic,category_turtleneck,category_vest
0,36,1,815826,10,20,334577,137,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,34,1,1636171,10,8,634115,125,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,0,438881,10,13,988705,124,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32,1,1392841,10,16,977884,158,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,34,2,160612,8,8,795673,135,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
#Concat the two datasets on index 
merged_df = pd.merge(numeric_cols, text_cols, how="outer", left_index=True, right_index=True)
merged_df.head()

age_x  fit_x  item_id  rating_x  size_x  user_id  weight_x  \
0     36      1   815826        10      20   334577       137   
1     34      1  1636171        10       8   634115       125   
2     30      0   438881        10      13   988705       124   
3     32      1  1392841        10      16   977884       158   
4     34      2   160612         8       8   795673       135   

   body_type_apple  body_type_athletic  body_type_full bust  \
0                0                   0                    0   
1                0                   1                    0   
2                0                   0                    0   
3                0                   0                    0   
4                0                   0                    0   

   body_type_hourglass  body_type_pear  body_type_petite  \
0                    1               0                 0   
1                    0               0                 0   
2                    1               0                 0   
3                    1               0                 0   
4                    0               1                 0   

   body_type_straight & narrow  rented_for_date  rented_for_everyday  \
0                            0                0                    0   
1                            0                0                    0   
2                            0                0                    0   
3                            0                0                    0   
4                            0                0                    0   

   rented_for_formal affair  rented_for_other  rented_for_party  \
0                         0                 0                 0   
1                         0                 0                 0   
2                         1                 0                 0   
3                         0                 0                 0   
4                         0                 0                 1   

   rented_for_vacation  rented_for_wedding  rented_for_work  \
0                    0                   1                0   
1                    0                   0                1   
2                    0                   0                0   
3                    0                   1                0   
4                    0                   0                0   

   category_ballgown  category_blazer  category_blouse  category_blouson  \
0                  0                0                0                 0   
1                  0                0                0                 0   
2                  0                0                0                 0   
3                  0                0                0                 0   
4                  0                0                0                 0   

   category_bomber  category_caftan  category_cami  category_cape  \
0                0                0              0              0   
1                0                0              0              0   
2                0                0              0              0   
3                0                0              0              0   
4                0                0              0              0   

   category_cardigan  category_coat  category_combo  category_culotte  \
0                  0              0               0                 0   
1                  0              0               0                 0   
2                  0              0               0                 0   
3                  0              0               0                 0   
4                  0              0               0                 0   

   category_culottes  category_down  category_dress  category_duster  \
0                  0              0               1                0   
1                  0              0               1                0   
2                  0              0               0                0   
3                  0              0               1       

In [8]:
merged_df.dropna(inplace= True)

In [9]:
merged_df.shape

(29221, 17219)

In [10]:
merged_df.isna().sum()

age_x                          0
fit_x                          0
item_id                        0
rating_x                       0
size_x                         0
user_id                        0
weight_x                       0
body_type_apple                0
body_type_athletic             0
body_type_full bust            0
body_type_hourglass            0
body_type_pear                 0
body_type_petite               0
body_type_straight & narrow    0
rented_for_date                0
rented_for_everyday            0
rented_for_formal affair       0
rented_for_other               0
rented_for_party               0
rented_for_vacation            0
rented_for_wedding             0
rented_for_work                0
category_ballgown              0
category_blazer                0
category_blouse                0
category_blouson               0
category_bomber                0
category_caftan                0
category_cami                  0
category_cape                  0
          

# Pre- Modeling

In [11]:
#Define X and y
X = merged_df.drop(columns= 'fit_x')
y = merged_df['fit_x']

In [12]:
#Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state =42, 
                                                    stratify = y)

In [13]:
# Check the baseline 
# We can see that we also have unbalanced classes 

y.value_counts(normalize = True)

1    0.735875
0    0.134561
2    0.129564
Name: fit_x, dtype: float64

In [14]:
# Function to run different model types
# Need to scale the data because some columns are numeric and others are binary 

def fit_and_evaluate(X, y, model_name='lr', model_type=LogisticRegression()):
    
    # Pipeline for pre-processing
    pipe = Pipeline([
        ('scale', StandardScaler()),
        (model_name, model_type)
    ])
    # Fit the model 
    model = pipe.fit(X, y)
    
    #Score
    score = model.score(X, y)
                   
    # Print attributes
    print(f"For model: {model_type}")
    print(f"Score: {score}")
    
    return model

# Models

In [ ]:
#The models for this large data set keep breaking the kernel. I will try to use AWS to compute these. 

In [ ]:
# Logistic Regression

logreg = fit_and_evaluate(X_train, y_train, model_name='logreg', model_type=LogisticRegression())

In [ ]:
# KNN

knn = fit_and_evaluate(X_train, y_train, model_name='knn', model_type=KNeighborsClassifier())

In [ ]:
# Random Forests

rf = fit_and_evaluate(X_train, y_train, model_name='rf', model_type=RandomForestClassifier())

In [ ]:
# Naive bayes

nb = fit_and_evaluate(X_train, y_train, model_name='nb', model_type=MultinomialNB())